# Analysing Dataset

In [1]:
import pandas as pd
import pandas_profiling
import requests
from functional import pseq
import pathlib
import os
import json
import time
import datetime

In [2]:
directory_liar_dataset = "../liar_dataset"
directory_statements = f"{directory_liar_dataset}/statements"
directory_visualizations = "../visualizations"

In [16]:
df = pd.concat([pd.read_csv(f"{directory_liar_dataset}/{part}.tsv", sep='\t', header=None) for part in ['train', 'valid']])
df.columns = ['statement_id', 'label', 'statement', 'subject', 'speaker', 'speakers_job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context']

df.statement_id = df.statement_id.apply(lambda x: x[:-5])  # remove .json and get just ID

In [4]:
df.head(10)

,statement_id,label,statement,subject,speaker,speakers_job_title,state_info,party_affiliation,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,context
0,2635,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
5,12465,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece
6,2342,barely-true,Jim Dunnam has not lived in the district he re...,candidates-biography,republican-party-texas,NaN,Texas,republican,3.0,1.0,1.0,3.0,1.0,a press release.
7,153,half-true,I'm the only person on this stage who has work...,ethics,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,"a Democratic debate in Philadelphia, Pa."
8,5602,half-true,"However, it took $19.5 million in Oregon Lotte...",jobs,oregon-lottery,NaN,NaN,organization,0.0,0.0,1.0,0.0,1.0,a website
9,9741,mostly-true,Says GOP primary opponents Glenn Grothman and ...,"energy,message-machine-2014,voting-record",duey-stroebel,State representative,Wisconsin,republican,0.0,0.0,0.0,1.0,0.0,an online video


In [5]:
# form URL from statement ID
def get_URL(statement_id):
    return f"http://www.politifact.com/api/v/2/statement/{statement_id}/?format=json"

In [6]:
def extract_information(res):
    try:
        author = res['author']

        try:
            if len(author) > 0:
                author = author[0]['name_slug']
            else:
                author = None
        except Exception:
            print(author)

        return {'author_name_slug': author,
                'ruling_date':  res['ruling_date'],
                'statement_date' :res['statement_date'],
                'speaker_current_job': res['speaker']['current_job'],
                'speaker_first_name': res['speaker']['first_name'],
                'speaker_last_name': res['speaker']['last_name'],
                'speaker_home_state': res['speaker']['home_state'],
                'statement_id': res['id']
               }
    except KeyError:
        return {}

In [7]:
#with requests.Session() as session:
#    additional_information = statement_ids.map(lambda sid: session.get(get_URL(sid)))\
#                                          .filter(lambda r: r.ok)\
#                                          .map(lambda r: r.json())\
#                                          .map(extract_information)\
#                                          .to_pandas()

In [8]:
def safe_json_read(f):
    try:
        with open(f, 'r') as fc:
            return json.load(fc)
    except json.JSONDecodeError:
        print(f)
        return {}

In [12]:
additional_information = pseq(pathlib.Path('../liar_dataset/statements/').iterdir())\
                               .map(safe_json_read)\
                               .filter(lambda x: len(x) > 0)\
                               .map(extract_information)\
                               .to_pandas()

additional_information['statement_date'] = pd.to_datetime(additional_information['statement_date'])

../liar_dataset/statements/9.json
../liar_dataset/statements/5355.json


In [197]:
def label_to_nb(l): 
    return ['true', 'mostly-true', 'half-true', 'barely-true', 'false', 'pants-fire'].index(l)

df['label_as_nb'] = df['label'].apply(label_to_nb) * 2 # TODO think about this, this will give the false-hoods more weight

In [198]:
df['statement_id'] = pd.to_numeric(df['statement_id'])
lies = df.merge(additional_information, on='statement_id', how='left')

In [199]:
lies.loc[lies['speaker'] == 'barack-obama', ]['pants_on_fire_counts'].value_counts()

9.0    549
Name: pants_on_fire_counts, dtype: int64

In [200]:
lies[lies['speakers_job_title'].str.contains('County') == True].shape

(265, 22)

In [201]:
lies['statement_date'].describe()

count                   11521
unique                   2832
top       2011-01-25 00:00:00
freq                       22
first     1995-04-01 00:00:00
last      2016-12-11 00:00:00
Name: statement_date, dtype: object

In [202]:
pandas_profiling.ProfileReport(lies)

Number of variables,22
Number of observations,11524
Total Missing (%),2.3%
Total size in memory,2.0 MiB
Average record size in memory,184.0 B
Numeric,4
Categorical,14
Boolean,0
Date,1
Text (Unique),0
Rejected,3


# federal election results

In [203]:
pd.options.display.max_colwidth = 300
pd.options.display.max_columns = 300

In [204]:
from itertools import product
from functools import reduce

In [205]:
def add_ending(f):
    if '2016' in f:
        return f"{f}x"
    else:
        return f
    
# TODO do 2012 it's a special snowflake
election_files = [(add_ending(f'../data/election_results/federalelections{year}.xls'), year) for year in [2014, 2016]]

In [206]:
election_results_cols_of_interest = ['CANDIDATE NAME', 'PRIMARY VOTES', 'PRIMARY %']

def fix_columns_election_results(df, year, type_):
    df = df.loc[:, election_results_cols_of_interest]
    df[f'primary_votes_{type_.lower()}_{year}'] = df['PRIMARY VOTES']
    df[f'primary_votes_{type_.lower()}_{year}_pct'] = df['PRIMARY %']
    return df.drop(columns=['PRIMARY VOTES', 'PRIMARY %'])


def get_only_voting_results(df):
    return df.loc[df['CANDIDATE NAME'].notna() & df['PRIMARY VOTES'].notna() & df['CANDIDATE NAME'].ne('Scattered') & df['CANDIDATE NAME'].ne('All Others'), :]


def prep_election_results(df, year, type_):
    return fix_columns_election_results(get_only_voting_results(df), year, type_)

In [207]:
election_results = [prep_election_results(pd.read_excel(f, sheet_name=f'{year} US {type_} Results by State'), year, type_) for (f, year), type_ in product(election_files, ['Senate', 'House'])]

election_results = reduce(lambda acc, el: pd.merge(acc, el, on='CANDIDATE NAME', how='outer'), election_results)

In [208]:
# yeah ... let's see how many we can join. the one letter endings might be a problem
election_results['CANDIDATE NAME'].value_counts()

Collins, Chris            12
Reed, Thomas W., II       12
Tonko, Paul D.            12
Maloney, Sean Patrick     12
Katko, John M.            12
Assini, Mark W.            8
Stefanik, Elise M.         8
Nadler, Jerrold L.         6
Crowley, Joseph            6
Slaughter, Louise M.       6
Engel, Eliot L.            6
Higgins, Brian             6
Lowey, Nita M.             6
Zeldin, Lee M.             6
Clarke, Yvette D.          4
Martin, Andy               4
Gregory, Du Wayne          4
Kuster, Ann McLane         4
Donovan, Dan               4
Jeffries, Hakeem S.        4
Schumer, Charles E.        4
Rice, Kathleen M.          4
Faso, John J.              4
Rubens, Jim                4
Maloney, Carolyn B.        4
Kelly, Walter W.           4
King, Peter T.             3
Long, Wendy                3
Oliva, Phil                3
Gurfein, David H.          3
                          ..
Hardin, Chris              1
Grier, Michael, Jr.        1
Hinz, Lynette "Moreno"     1
Macko, David  

In [209]:
# we are only interest in people and they have a first name
lies = lies.loc[lies['speaker_first_name'].notnull(), :]

In [210]:
# to aggregate the statements
lies['statement_year'] = lies['statement_date'].dt.year

# for the merging
lies['speaker_full_name'] = lies['speaker_last_name'] + ', ' + lies['speaker_first_name']

In [211]:
# todo expand this and check this! this is just a quick and dirty fix
# is it really houseman? probably not...
_job_titles_of_interest = [('senat', 'senator'), ('governor', None), ('congress', 'congressman'), ('mayor', None), ('president', None), ('house', 'houseman'), ('rep', 'houseman')]
job_titles_of_interest = [out if out is not None else j for j, out in _job_titles_of_interest]

def cleaned_job_title(jt):
    jt = str(jt).lower()
    
    for j, out in _job_titles_of_interest:
        if j in jt:
            return out if out is not None else j
    else:
        return jt

lies['speakers_job_title_cleaned'] = lies['speakers_job_title'].apply(cleaned_job_title)

In [212]:
_t = lies.merge(election_results, left_on='speaker_full_name', right_on='CANDIDATE NAME', how='outer')

In [213]:
print(f"found election results for {_t['CANDIDATE NAME'].notnull().sum()} ({_t['CANDIDATE NAME'].notnull().mean()}%) people")

found election results for 4952 (0.328403740301081%) people


In [214]:
votes_cols = [c for c in _t.columns if 'votes' in c]
useful_idx = reduce(lambda acc, el: acc | el, [_t[c].notnull() for c in votes_cols]) & _t['speaker'].notnull()

print(f"found useful results for {useful_idx.sum()} people")

columns_of_interest = ['label', 'label_as_nb', 'subject', 'speaker', 'speakers_job_title_cleaned', 'state_info', 'party_affiliation', 'context', 'statement_date'] + votes_cols
_t.loc[useful_idx, columns_of_interest]

found useful results for 1461 people


,label,label_as_nb,subject,speaker,speakers_job_title_cleaned,state_info,party_affiliation,context,statement_date,primary_votes_senate_2014,primary_votes_senate_2014_pct,primary_votes_house_2014,primary_votes_house_2014_pct,primary_votes_senate_2016,primary_votes_senate_2016_pct,primary_votes_house_2016,primary_votes_house_2016_pct
1115,half-true,4.0,"financial-regulation,foreign-policy,water",gwen-moore,houseman,Wisconsin,democrat,a congressional hearing,2016-04-27,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1116,mostly-true,2.0,"crime,criminal-justice,women",gwen-moore,houseman,Wisconsin,democrat,a letter to congressional leadership,2012-12-11,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1117,barely-true,6.0,"city-government,civil-rights,criminal-justice,public-safety,state-budget",gwen-moore,houseman,Wisconsin,democrat,comments to reporters,2015-08-26,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1118,true,0.0,"city-government,infrastructure,transportation",gwen-moore,houseman,Wisconsin,democrat,a newspaper opinion piece,2015-01-14,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1119,false,8.0,"children,crime,criminal-justice,sexuality",gwen-moore,houseman,Wisconsin,democrat,remarks on the House floor,2014-05-29,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1120,pants-fire,10.0,"corrections-and-updates,health-care,women",gwen-moore,houseman,Wisconsin,democrat,tweets,2011-12-27,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1121,true,0.0,"civil-rights,crime,criminal-justice,legal-issues",gwen-moore,houseman,Wisconsin,democrat,an interview,2016-11-18,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1122,mostly-true,2.0,"crime,guns,terrorism,transportation",gwen-moore,houseman,Wisconsin,democrat,a tweet,2016-06-23,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1123,barely-true,6.0,"economy,health-care,medicaid,medicare",gwen-moore,houseman,Wisconsin,democrat,a radio interview,2014-11-11,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1124,false,8.0,"labor,state-budget,state-finances",gwen-moore,houseman,Wisconsin,democrat,a speech at a state Democratic Party dinner,2011-04-30,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN


In [215]:
_t.loc[useful_idx, 'speakers_job_title_cleaned'].value_counts()

senator                                                                                                 580
houseman                                                                                                353
milwaukee county executive                                                                              166
congressman                                                                                             111
nan                                                                                                     101
governor                                                                                                 41
businessman                                                                                              13
small business owner                                                                                     13
retired social worker                                                                                     8
state assembly member, 78th 

In [216]:
_t.loc[_t['speakers_job_title_cleaned'].isin(job_titles_of_interest) & useful_idx, columns_of_interest]

,label,label_as_nb,subject,speaker,speakers_job_title_cleaned,state_info,party_affiliation,context,statement_date,primary_votes_senate_2014,primary_votes_senate_2014_pct,primary_votes_house_2014,primary_votes_house_2014_pct,primary_votes_senate_2016,primary_votes_senate_2016_pct,primary_votes_house_2016,primary_votes_house_2016_pct
1115,half-true,4.0,"financial-regulation,foreign-policy,water",gwen-moore,houseman,Wisconsin,democrat,a congressional hearing,2016-04-27,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1116,mostly-true,2.0,"crime,criminal-justice,women",gwen-moore,houseman,Wisconsin,democrat,a letter to congressional leadership,2012-12-11,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1117,barely-true,6.0,"city-government,civil-rights,criminal-justice,public-safety,state-budget",gwen-moore,houseman,Wisconsin,democrat,comments to reporters,2015-08-26,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1118,true,0.0,"city-government,infrastructure,transportation",gwen-moore,houseman,Wisconsin,democrat,a newspaper opinion piece,2015-01-14,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1119,false,8.0,"children,crime,criminal-justice,sexuality",gwen-moore,houseman,Wisconsin,democrat,remarks on the House floor,2014-05-29,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1120,pants-fire,10.0,"corrections-and-updates,health-care,women",gwen-moore,houseman,Wisconsin,democrat,tweets,2011-12-27,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1121,true,0.0,"civil-rights,crime,criminal-justice,legal-issues",gwen-moore,houseman,Wisconsin,democrat,an interview,2016-11-18,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1122,mostly-true,2.0,"crime,guns,terrorism,transportation",gwen-moore,houseman,Wisconsin,democrat,a tweet,2016-06-23,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1123,barely-true,6.0,"economy,health-care,medicaid,medicare",gwen-moore,houseman,Wisconsin,democrat,a radio interview,2014-11-11,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1124,false,8.0,"labor,state-budget,state-finances",gwen-moore,houseman,Wisconsin,democrat,a speech at a state Democratic Party dinner,2011-04-30,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN


# DATA SET COMPLETE

In [219]:
median_speaker_value = _t.groupby(['statement_year', 'speaker'])['label_as_nb'].median().reset_index()

In [220]:
median_speaker_value[median_speaker_value['statement_year'] == 2016]

,statement_year,speaker,label_as_nb
4334,2016.0,18-percent-american-public,0.0
4335,2016.0,Jack_Graham,8.0
4336,2016.0,aclu-north-carolina,0.0
4337,2016.0,actionaid-k,3.0
4338,2016.0,afscme,4.0
4339,2016.0,afscme-people,2.0
4340,2016.0,al-gore,4.0
4341,2016.0,alan-grayson,3.0
4342,2016.0,alcee-hastings,10.0
4343,2016.0,alex-castellanos,6.0


## One row analysis

Let's analyse first row, statement with id `1`. What is the information we get there?

In [17]:
sid = '1'

In [18]:
df[df.statement_id == sid]

,statement_id,label,statement,subject,speaker,speakers_job_title,state_info,party_affiliation,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,context
9403,1,pants-fire,The attorney general requires that rape victim...,"crime,women",barbara-ann-radnofsky,NaN,NaN,democrat,0.0,0.0,0.0,0.0,1.0,in a Web site video


In [19]:
with open(f"{directory_statements}/{sid}.json", "r") as f:
    data = json.load(f)
data

{'art': [{'brightcove': '',
   'caption': "We're Going to Need a Bigger List",
   'id': 257,
   'infogram': '',
   'ndn': '',
   'ndnid': '',
   'other': '',
   'photo': None,
   'resource_type': {'id': 2,
    'name': 'YouTube',
    'resource_uri': '/api/v/2/mediatype/2/'},
   'resource_uri': '/api/v/2/media/257/',
   'title': 'Radnofsky video',
   'wibbitz': '',
   'youtube': '<iframe src="//www.youtube.com/embed/OrKDN_FL4iM?rel=0&wmode=opaque" frameborder="0" allowfullscreen></iframe>',
   'youtubeID': 'OrKDN_FL4iM'}],
 'author': [{'email_address': 'mashford-grooms@statesman.com',
   'first_name': 'Meghan',
   'id': 118,
   'last_name': 'Ashford-Grooms',
   'name_slug': 'meghan-ashford-grooms',
   'on_staff_page': None,
   'phone': None,
   'photo': None,
   'publication': {'id': 3,
    'publication_name': 'Austin American-Statesman',
    'resource_uri': '/api/v/2/publication/3/'},
   'resource_uri': '/api/v/2/staffer/118/',
   'title': 'PolitiFact Texas staff writer',
   'twitter': 

Just to visualize JSON hierarchy, run the following cell:

In [38]:
def go_further(dic, name):
    dict_vis = {"name": name, "children": []}
    for k, v in dic.items():
        if type(v) == str:
            new_el = {"name": k}
        elif type(v) == list:
            if len(v) > 0:
                new_el = go_further(v[0], k)
        elif type(v) == dict:
            new_el = go_further(v, k)
        else:
            new_el = {"name": k}
        dict_vis["children"].append(new_el)
        
    return dict_vis

my_dict = go_further(data, name="statement_info")

with open(f"{directory_visualizations}/data.json", "w") as f:
    json.dump(my_dict, f)

print(f"Checkout visualization by: \n1) cd ../visualizations \n2) python -m http.server \n3) in browser, open: http://localhost:8000/")

Checkout visualization by: 
1) cd ../visualizations 
2) python -m http.server 
3) in browser, open: http://localhost:8000/
